In [19]:
import numpy as np
import utils
from utils import loadmat
from extractDigitFeatures import extractDigitFeatures
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, UpSampling2D, Flatten, MaxPool2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
from keras import regularizers
from keras.constraints import unit_norm
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping

In [4]:
def get_random_data(data1, data2, low, high, max_samples=100):
    N, H1, W1, C1 = data1.shape
    #_, N1 = data2.shape
    suff_data1 = np.zeros((max_samples, H1, W1, C1))
    suff_data2 = np.zeros((max_samples,))
    shuffles = np.random.randint(low, high+1, max_samples)
    for idx in range(shuffles.shape[0]):
        suff_data1[idx] = data1[idx, :, :, :]
        suff_data2[idx] = data2[idx]
    return suff_data1, suff_data2

def reformat(x):
    new_x = np.zeros((x.shape[2], x.shape[0], x.shape[1]))
    for i in range(x.shape[2]):
        new_x[i, :] = x[:, :, i]
    return new_x[:, :, :, np.newaxis]

def return_datasets(filename):
    data = utils.loadmat('../data/{}'.format(filename))
    trainSet, testSet, valSet = 1, 2, 3
    
    x_train = reformat(data['x'][:, :, data['set']==trainSet ])
    y_train = (data['y'][data['set']==trainSet])
    x_val = reformat(data['x'][:, :, data['set']==valSet])
    y_val = (data['y'][data['set']==valSet])
    x_test = reformat(data['x'][:, :, data['set']==testSet])
    y_test = (data['y'][data['set']==testSet])
    
    x_train, y_train = get_random_data(x_train, y_train, 0, x_train.shape[0], x_train.shape[0])
    x_val, y_val = get_random_data(x_val, y_val, 0, x_val.shape[0], x_val.shape[0])
    x_test, y_test = get_random_data(x_test, y_test, 0, x_test.shape[0], x_test.shape[0])
    
    #print (x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)
    

    #return (X, Y, x_test, y_test)
    return (x_train, y_train, x_val, y_val, x_test, y_test)

In [32]:
def extract_model(activation_func = 'relu'):
    model = Sequential()
    kernel_size=(5)
    stride_size = (2)

    model.add(Conv2D(32, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same', input_shape=(28, 28, 1)))
    model.add(MaxPool2D(2, 2))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same'))
    model.add(MaxPool2D(2, 2))
    model.add(BatchNormalization())
    
    model.add(UpSampling2D((2, 2)))
#     model.add(Conv2D(64, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same'))
#     model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same'))
    model.add(MaxPool2D(2, 2))
    model.add(BatchNormalization())
    
    
    model.add(Conv2D(96, kernel_size=kernel_size, strides=stride_size, activation=activation_func, padding='same'))
    model.add(BatchNormalization())


    model.add(Flatten())
    model.add(Dense(256, activation=activation_func)) 
    model.add(Dense(10, activation='softmax'))
    
    return model

In [21]:
MAX_EPOCHS = 30
MAX_BATCH_SIZE = 25

In [33]:
file = 'digits-normal.mat'
x_train, y_train, x_val, y_val, x_test, y_test = return_datasets(file)

model = extract_model('relu')
model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(x_val, to_categorical(y_val, num_classes=10)), metrics=["accuracy"])
model.fit(x_train, to_categorical(y_train, num_classes=10), epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE, verbose=1)
accuracy = model.evaluate(x_test, to_categorical(y_test, num_classes=10), verbose=0)[1]
print (accuracy)

Train on 1000 samples
Epoch 1/30
1000/1000 [==============================] - 35s 35ms/sample - loss: 1.0754 - accuracy: 0.6800
Epoch 2/30
1000/1000 [==============================] - 0s 366us/sample - loss: 0.3140 - accuracy: 0.9200
Epoch 3/30
1000/1000 [==============================] - 0s 390us/sample - loss: 0.1400 - accuracy: 0.9670
Epoch 4/30
1000/1000 [==============================] - 0s 386us/sample - loss: 0.0832 - accuracy: 0.9810
Epoch 5/30
1000/1000 [==============================] - 0s 382us/sample - loss: 0.0760 - accuracy: 0.9790
Epoch 6/30
1000/1000 [==============================] - 0s 382us/sample - loss: 0.0633 - accuracy: 0.9820
Epoch 7/30
1000/1000 [==============================] - 0s 393us/sample - loss: 0.0985 - accuracy: 0.9690
Epoch 8/30
1000/1000 [==============================] - 0s 392us/sample - loss: 0.1034 - accuracy: 0.9670
Epoch 9/30
1000/1000 [==============================] - 0s 386us/sample - loss: 0.0551 - accuracy: 0.9860
Epoch 10/30
1000/1000 [=

In [26]:
file = 'digits-scaled.mat'
x_train, y_train, x_val, y_val, x_test, y_test = return_datasets(file)

model = extract_model('selu')
model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(x_val, to_categorical(y_val, num_classes=10)), metrics=["accuracy"])
model.fit(x_train, to_categorical(y_train, num_classes=10), epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE, verbose=1)
accuracy = model.evaluate(x_test, to_categorical(y_test, num_classes=10), verbose=0)[1]
print (accuracy)

Train on 1000 samples
Epoch 1/30
1000/1000 [==============================] - 10s 10ms/sample - loss: 1.6925 - accuracy: 0.4300
Epoch 2/30
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.9311 - accuracy: 0.7010
Epoch 3/30
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.7029 - accuracy: 0.7730
Epoch 4/30
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.6725 - accuracy: 0.7930
Epoch 5/30
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.5581 - accuracy: 0.8100
Epoch 6/30
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.5000 - accuracy: 0.8420
Epoch 7/30
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.4271 - accuracy: 0.8620
Epoch 8/30
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.3118 - accuracy: 0.9040
Epoch 9/30
1000/1000 [==============================] - 1s 1ms/sample - loss: 0.3431 - accuracy: 0.8880
Epoch 10/30
1000/1000 [=================

In [27]:
accuracy

0.836

In [13]:
file = 'digits-scaled.mat'
x_train, y_train, x_val, y_val, x_test, y_test = return_datasets(file)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
best_model, best_accuracy = -1, -1

for train, test in kfold.split(x_train, y_train):
    curr_x_train, curr_y_train = x_train[train], to_categorical(y_train[train], num_classes=10)
    curr_x_val, curr_y_val = x_train[train], to_categorical(y_train[train], num_classes=10)
    
    model = extract_model()
    if file == 'digits-scaled.mat':
        model = extract_model('selu')

    model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(curr_x_val, curr_y_val), metrics=["accuracy"])
    model.fit(curr_x_train, curr_y_train, epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE, verbose=1)
    accuracy = model.evaluate(x_val, to_categorical(y_val, num_classes=10), verbose=0)[1]
    print ("Accuracy = {}".format(accuracy))
    if accuracy > best_accuracy:
        best_accuracy, best_model = accuracy, model

print ("Test Set Accuracy for {} = {}".format(file, best_model.evaluate(x_test, to_categorical(y_test, num_classes=10), verbose=0)))
    
    

Train on 900 samples
Epoch 1/30
900/900 [==============================] - 56s 62ms/sample - loss: 1.8467 - accuracy: 0.3867
Epoch 2/30
900/900 [==============================] - 1s 655us/sample - loss: 1.0552 - accuracy: 0.6522
Epoch 3/30
900/900 [==============================] - 0s 531us/sample - loss: 0.8165 - accuracy: 0.7489
Epoch 4/30
900/900 [==============================] - 0s 518us/sample - loss: 0.6959 - accuracy: 0.7722
Epoch 5/30
900/900 [==============================] - 0s 534us/sample - loss: 0.6550 - accuracy: 0.7922
Epoch 6/30
900/900 [==============================] - 1s 597us/sample - loss: 0.5903 - accuracy: 0.8133
Epoch 7/30
900/900 [==============================] - 0s 516us/sample - loss: 0.4454 - accuracy: 0.8700
Epoch 8/30
900/900 [==============================] - 0s 514us/sample - loss: 0.3975 - accuracy: 0.8767
Epoch 9/30
900/900 [==============================] - 0s 522us/sample - loss: 0.4030 - accuracy: 0.8789
Epoch 10/30
900/900 [======================

KeyboardInterrupt: 